[Census API Datasets]('https://api.census.gov/data.html')
* ACS/Tiger Mapping files: https://www2.census.gov/geo/tiger/TIGER_DP/
* 2022 Relationship Files: https://www2.census.gov/geo/tiger/TIGER2022/

In [1]:
import pandas 
import json
import requests
import numpy
import os
import sys
import geopandas
import janitor
from path import Path

In [13]:
# path configuration
working_directory = Path.getcwd()

inputs_path = working_directory / 'inputs'
outputs_path = working_directory / 'outputs'

In [14]:
# export function
def to_file(callback, desired_name, extension='csv'):
    current_files = sorted(outputs_path.files(desired_name + '*.' + extension))
    if current_files:
        last_file = current_files[-1]
        os.remove(last_file)
    final_name = '{}.{}'.format(desired_name, extension)
    callback(outputs_path / final_name)

In [2]:
# capture geoid codes from url
geoid_codes = 'https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt'

# read in url and retain fips codes and names, set as string objects and index
state_fips = pandas.read_fwf(geoid_codes,skiprows=16,nrows=53,names=['FIPS Code','Name'],dtype={'FIPS Code': 'str'})
county_fips = pandas.read_fwf(geoid_codes,skiprows=73,names=['FIPS Code','Name'],dtype={'FIPS Code': 'str'})

# Define a function to apply str.title()
def title_col(df, column_name):
    df[column_name] = df[column_name].str.title()

# apply function and rename with appropriate geos
title_col(state_fips, 'Name')
title_col(county_fips, 'Name')

# rename column fips codes to appropriate geos

state_fips.rename(columns={'FIPS Code':'state'},inplace=True)
county_fips.rename(columns={'FIPS Code':'county'},inplace=True)

In [3]:
# make dictionaries
state_dict = state_fips.set_index('state')['Name'].to_dict()
county_dict = county_fips.set_index('county')['Name'].to_dict()

### Example API

In [4]:
base_url = "https://api.census.gov/data"
dataset = "acs/acs5"
year = "2019"
variables = "B01001_001E"  # Population variable
state = "29"  # California FIPS code
county = "037"  # Los Angeles County FIPS code
tract = "*"  # All tracts within the county
api_key = "84c7d937e5b6038afeb17bd5615eceef179ff9a3"

endpoint = f"{base_url}/{year}/{dataset}?get={variables}&for=tract:{tract}&in=state:{state}&in=county:{county}&key={api_key}"

In [5]:
# set up response for API 

response = requests.get(endpoint)
if response.status_code == 200:
    data = response.json()
else:
    print("Error fetching data from the API")

In [6]:
# create dataframe 
df = pandas.DataFrame(data[1:], columns=data[0])

# make geograpihes a string
df[['state','county','tract']] = df[['state','county','tract']].astype('str')

In [8]:
# view
df.head()

,B01001_001E,state,county,tract
0,10552,29,037,060301
1,5079,29,037,060400
2,3968,29,037,061200
3,8156,29,037,060302
4,4316,29,037,061100


In [9]:
# add prefix to county name
df['county'] = df['state'] + df['county']

# append  dictionary
df = df.replace({'county':county_dict,
                 'state':state_dict})

# change table name
df.rename(columns={'B01001_001E':'total_population'},inplace= True)

In [15]:
to_file(df.to_csv, 'CensusTractPop_CassCounty')

In [ ]:
# You can save the results to a file or process them as needed

### Example 2 - More Detailed Table Collection

In [229]:
# read in product table and create different api call

# path configuration
working_directory = Path.getcwd()
inputs_path = working_directory /'inputs'

# read in spreadsheet with table names
tables = pandas.read_excel(inputs_path/'2021_DataProductList.xlsx')

In [232]:
tables.head()

,Table ID,Table Title,Table Universe,Data Product Type,Year,1-Year Geography Restrictions\n(with Summary Levels in Parentheses),5-Year Geography Restrictions\n(with Summary Levels in Parentheses)
0,B01001,SEX BY AGE,Total population,Detailed Table,"1,5",NaN,NaN
1,B01001A,SEX BY AGE (WHITE ALONE),People who are White alone,Detailed Table,"1,5",NaN,Excludes Place/Remainder and Block Group geogr...
2,C01001A,SEX BY AGE (WHITE ALONE),People who are White alone,Detailed Table,1,NaN,NaN
3,B01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE),People who are Black or African American alone,Detailed Table,"1,5",NaN,Excludes Place/Remainder and Block Group geogr...
4,C01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE),People who are Black or African American alone,Detailed Table,1,NaN,NaN


In [267]:
# filter to see if there are table dems of interest

table_dems = list(tables['Table Universe'].unique())

# example - use keywords transportation and income and filter for table names

# get all tables for C0100A
group = list(tables[tables['Table ID'].str.contains('C01001')]['Table ID'].unique())

In [268]:
group

['C01001A',
 'C01001B',
 'C01001C',
 'C01001D',
 'C01001E',
 'C01001F',
 'C01001G',
 'C01001H',
 'C01001I']

In [277]:
# different API call - all sex by age

base_url = "https://api.census.gov/data"
dataset = "acs/acs5"
year = "2019"
variables = group  # Population variable
state = "06"  # California FIPS code
county = "073"  # Los Angeles County FIPS code
tract = "*"  # All tracts within the county
api_key = "84c7d937e5b6038afeb17bd5615eceef179ff9a3"

endpoint = f"{base_url}/{year}/{dataset}?get=group(B02015)&for=tract:{tract}&in=state:{state}&in=county:{county}&key={api_key}"